# microWakeWord
This notebook is based on basic_training_notebook.ipynb from microWakeWord from here: https://github.com/kahrendt/microWakeWord/blob/main/notebooks/basic_training_notebook.ipynb.
And hausklaus from here: https://github.com/ABee81/hausklaus-mww/blob/main/hausklaus-mww.ipynb

Datasets were extended, configured properly for mls model and training parameters were changed for longer training.
Executing this notebook step by step will create a stream_state_internal_quant.tflite file in data/trained_models/mww/tflite_stream_state_internal_quant.
This model can then be used in esphome on the HA VPE for an on device wakeword detection.


## Installation

In [ ]:
# Install microWakeWord
!pip install -e ./microWakeWord

# # Download mls model for de_DE
# !wget -O piper-sample-generator/models/de_DE-mls-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/de_DE-mls-medium.pt'

!wget -O piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'


In [ ]:
import sys
sys.path.append("./piper-sample-generator/")
sys.path

In [ ]:
from generate_samples import generate_samples
from microwakeword.audio.augmentation import Augmentation
from microwakeword.audio.clips import Clips
from microwakeword.audio.spectrograms import SpectrogramGeneration
from microwakeword.audio.audio_utils import save_clip
from mmap_ninja.ragged import RaggedMmap
from IPython.display import Audio
import datasets
import scipy
import numpy as np
from pathlib import Path
from tqdm import tqdm
import yaml
import os
import shutil

## Settings

In [ ]:
WAKEWORD = "hey eve"
SAMPLE_COUNT = 4096
BATCH_SIZE = 16

## Sample generation

In [ ]:

generate_samples(
    text=WAKEWORD,
    output_dir="./data/samples",
    max_samples=1,
    batch_size=BATCH_SIZE,
    model="piper-sample-generator/models/en_US-libritts_r-medium.pt",
    slerp_weights=[0.5],
    length_scales=[1.0, 0.75, 1.25, 1.4],
    noise_scales=[0.333],
    noise_scale_ws=[0.333],
    min_phoneme_count=80,
    verbose=True
)

Audio("./data/samples/0.wav", autoplay=True)

In [ ]:
generate_samples(
    text=WAKEWORD,
    output_dir="./data/samples",
    max_samples=SAMPLE_COUNT,
    batch_size=BATCH_SIZE,
    model="piper-sample-generator/models/en_US-libritts_r-medium.pt",
    slerp_weights=[0.5],
    length_scales=[1.0, 0.75, 1.25, 1.4],
    noise_scales=[0.333],
    noise_scale_ws=[0.333],
    min_phoneme_count=80
)


## Dataset Preparation

### MIT RIR Dataset

In [ ]:

output_dir = "./data/mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    rir_dataset = datasets.load_dataset("davidscripka/MIT_environmental_impulse_responses", split="train", streaming=True)
    # Save clips to 16-bit PCM wav files
    for row in tqdm(rir_dataset):
        name = row['audio']['path'].split('/')[-1]
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))


### AudioSet Dataset

In [ ]:
# AudioSet Dataset (https://research.google.com/audioset/dataset/index.html)

if not os.path.exists("data/audioset"):
    os.mkdir("data/audioset")


    for i in range(10):
        fname = f"bal_train0{i}.tar"
        out_dir = f"audioset/{fname}"
        link = "https://huggingface.co/datasets/agkphysics/AudioSet/resolve/main/data/" + fname
        !wget -O {out_dir} {link}
        !cd audioset && tar -xf bal_train0*.tar

    output_dir = "./data/audioset_16k"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Save clips to 16-bit PCM wav files
    audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path("data/audioset/audio").glob("**/*.flac")]})
    audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
    for row in tqdm(audioset_dataset):
        name = row['audio']['path'].split('/')[-1].replace(".flac", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))


### fma_small Dataset

In [ ]:
output_dir = "./data/fma_16k"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

    # iterate over each subdirectory in the fma_small directory
    if not os.path.exists("./data/fma_small"):
        !wget -O fma_small.zip https://os.unil.cloud.switch.ch/fma/fma_small.zip
        !unzip fma_small.zip -d fma_small/
    
    # Save clips to 16-bit PCM wav files
    fma_dataset = datasets.Dataset.from_dict({"audio": [str(i) for dir in os.listdir("data/fma_small") for i in Path(f"data/fma_small/{dir}").glob("**/**/*.mp3")]})
    fma_dataset = fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
    
    for row in tqdm(fma_dataset):
        name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
        if not os.path.exists(os.path.join(output_dir, name)):
            # Convert to 16-bit PCM wav format
            scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))



### Augmentation
Most of this part is from kahrendt/microWakeWord: https://github.com/kahrendt/microWakeWord/blob/main/notebooks/basic_training_notebook.ipynb 

Only parameters were changed where needed

In [ ]:
# Sets up the augmentations.
# To improve your model, experiment with these settings and use more sources of
# background clips.

clips = Clips(input_directory='data/samples',
              file_pattern='*.wav',
              max_clip_duration_s=None,
              remove_silence=False,
              random_split_seed=10,
              split_count=0.1,
              )
augmenter = Augmentation(augmentation_duration_s=3.2,
                         augmentation_probabilities = {
                                "SevenBandParametricEQ": 0.1,
                                "TanhDistortion": 0.1,
                                "PitchShift": 0.1,
                                "BandStopFilter": 0.1,
                                "AddColorNoise": 0.1,
                                "AddBackgroundNoise": 0.75,
                                "Gain": 1.0,
                                "RIR": 0.5,
                            },
                         impulse_paths = ['data/mit_rirs'],
                         background_paths = ['data/fma_16k', 'data/audioset_16k'],
                         background_min_snr_db = -5,
                         background_max_snr_db = 10,
                         min_jitter_s = 0.195,
                         max_jitter_s = 0.205,
                         )


In [ ]:

# Augment a random clip and play it back to verify it works well

random_clip = clips.get_random_clip()
augmented_clip = augmenter.augment_clip(random_clip)
save_clip(augmented_clip, 'augmented_clip.wav')

Audio("augmented_clip.wav", autoplay=True)

In [ ]:
# Augment samples and save the training, validation, and testing sets.
# Validating and testing samples generated the same way can make the model
# benchmark better than it performs in real-word use. Use real samples or TTS
# samples generated with a different TTS engine to potentially get more accurate
# benchmarks.

output_dir = 'data/augmented_features'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

splits = ["training", "validation", "testing"]
for split in splits:
  out_dir = os.path.join(output_dir, split)
  if not os.path.exists(out_dir):
      os.mkdir(out_dir)


  split_name = "train"
  repetition = 2

  spectrograms = SpectrogramGeneration(clips=clips,
                                     augmenter=augmenter,
                                     slide_frames=10,    # Uses the same spectrogram repeatedly, just shifted over by one frame. This simulates the streaming inferences while training/validating in nonstreaming mode.
                                     step_ms=10,
                                     )
  if split == "validation":
    split_name = "validation"
    repetition = 1
  elif split == "testing":
    split_name = "test"
    repetition = 1
    spectrograms = SpectrogramGeneration(clips=clips,
                                     augmenter=augmenter,
                                     slide_frames=1,    # The testing set uses the streaming version of the model, so no artificial repetition is necessary
                                     step_ms=10,
                                     )

  RaggedMmap.from_generator(
      out_dir=os.path.join(out_dir, 'wakeword_mmap'),
      sample_generator=spectrograms.spectrogram_generator(split=split_name, repeat=repetition),
      batch_size=7,
      verbose=True,
  )

### Download Spectogram feautures

In [ ]:
# Downloads pre-generated spectrogram features (made for microWakeWord in
# particular) for various negative datasets. This can be slow!

output_dir = './data/negative_datasets'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    link_root = "https://huggingface.co/datasets/kahrendt/microwakeword/resolve/main/"
    filenames = ['dinner_party.zip', 'dinner_party_eval.zip', 'no_speech.zip', 'speech.zip']
    for fname in filenames:
        link = link_root + fname

        zip_path = f"data/negative_datasets/{fname}"
        !wget -O {zip_path} {link}
        !unzip -q {zip_path} -d {output_dir}

## Training

In [ ]:
# Save a yaml config that controls the training process
# These hyperparamters can make a huge different in model quality.
# Experiment with sampling and penalty weights and increasing the number of
# training steps.

config = {}

config["window_step_ms"] = 10

config["train_dir"] = (
    f"data/trained_models/{WAKEWORD.replace(',', '')}"  # Directory where the trained model will be saved
)


# Each feature_dir should have at least one of the following folders with this structure:
#  training/
#    ragged_mmap_folders_ending_in_mmap
#  testing/
#    ragged_mmap_folders_ending_in_mmap
#  testing_ambient/
#    ragged_mmap_folders_ending_in_mmap
#  validation/
#    ragged_mmap_folders_ending_in_mmap
#  validation_ambient/
#    ragged_mmap_folders_ending_in_mmap
#
#  sampling_weight: Weight for choosing a spectrogram from this set in the batch
#  penalty_weight: Penalizing weight for incorrect predictions from this set
#  truth: Boolean whether this set has positive samples or negative samples
#  truncation_strategy = If spectrograms in the set are longer than necessary for training, how are they truncated
#       - random: choose a random portion of the entire spectrogram - useful for long negative samples
#       - truncate_start: remove the start of the spectrogram
#       - truncate_end: remove the end of the spectrogram
#       - split: Split the longer spectrogram into separate spectrograms offset by 100 ms. Only for ambient sets

config["features"] = [
    {
        "features_dir": "data/augmented_features",
        "sampling_weight": 2.0,
        "penalty_weight": 1.0,
        "truth": True,
        "truncation_strategy": "truncate_start",
        "type": "mmap",
    },
    {
        "features_dir": "data/negative_datasets/speech",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    {
        "features_dir": "data/negative_datasets/dinner_party",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    {
        "features_dir": "data/negative_datasets/no_speech",
        "sampling_weight": 5.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    { # Only used for validation and testing
        "features_dir": "data/negative_datasets/dinner_party_eval",
        "sampling_weight": 0.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "split",
        "type": "mmap",
    },
]

# Number of training steps in each iteration - various other settings are configured as lists that corresponds to different steps
config["training_steps"] = [100000]  # Number of training steps for each training iteration - list that corresponds to training steps

# Penalizing weight for incorrect class predictions - lists that correspond to training steps
config["positive_class_weight"] = [1]
config["negative_class_weight"] = [20]

config["learning_rates"] = [
    0.001,
]  # Learning rates for Adam optimizer - list that corresponds to training steps
config["batch_size"] = BATCH_SIZE

config["time_mask_max_size"] = [
    0
]  # SpecAugment - list that corresponds to training steps
config["time_mask_count"] = [0]  # SpecAugment - list that corresponds to training steps
config["freq_mask_max_size"] = [
    0
]  # SpecAugment - list that corresponds to training steps
config["freq_mask_count"] = [0]  # SpecAugment - list that corresponds to training steps

config["eval_step_interval"] = (
    2500  # Test the validation sets after every this many steps
)
config["clip_duration_ms"] = (
    1500  # Maximum length of wake word that the streaming model will accept
)

# The best model weights are chosen first by minimizing the specified minimization metric below the specified target_minimization
# Once the target has been met, it chooses the maximum of the maximization metric. Set 'minimization_metric' to None to only maximize
# Available metrics:
#   - "loss" - cross entropy error on validation set
#   - "accuracy" - accuracy of validation set
#   - "recall" - recall of validation set
#   - "precision" - precision of validation set
#   - "false_positive_rate" - false positive rate of validation set
#   - "false_negative_rate" - false negative rate of validation set
#   - "ambient_false_positives" - count of false positives from the split validation_ambient set
#   - "ambient_false_positives_per_hour" - estimated number of false positives per hour on the split validation_ambient set
config["target_minimization"] = 0.9
config["minimization_metric"] = None  # Set to None to disable

config["maximization_metric"] = "average_viable_recall"

with open(os.path.join("training_parameters.yaml"), "w") as file:
    documents = yaml.dump(config, file)

In [ ]:
# Trains a model. When finished, it will quantize and convert the model to a
# streaming version suitable for on-device detection.
# It will resume if stopped, but it will start over at the configured training
# steps in the yaml file.
# Change --train 0 to only convert and test the best-weighted model.
# On Google colab, it doesn't print the mini-batch results, so it may appear
# stuck for several minutes! Additionally, it is very slow compared to training
# on a local GPU.

!python -m microwakeword.model_train_eval \
--training_config='training_parameters.yaml' \
--train 1 \
--restore_checkpoint 1 \
--test_tf_nonstreaming 0 \
--test_tflite_nonstreaming 0 \
--test_tflite_nonstreaming_quantized 0 \
--test_tflite_streaming 0 \
--test_tflite_streaming_quantized 1 \
--use_weights "best_weights" \
mixednet \
--pointwise_filters "64,64,64,64" \
--repeat_in_block  "1, 1, 1, 1" \
--mixconv_kernel_sizes '[5], [7,11], [9,15], [23]' \
--residual_connection "0,0,0,0" \
--first_conv_filters 32 \
--first_conv_kernel_size 5 \
--stride 3

## Deploy

In [ ]:
# Move the model to the models directory

shutil.move(f"data/trained_models/{WAKEWORD.replace(',', '')}/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite", f"model/{WAKEWORD.replace(',', '')}.tflite")
shutil.move(f"data/trained_models/{WAKEWORD.replace(',', '')}/tflite_stream_state_internal_quant/tflite_streaming_roc.txt", f"model/{WAKEWORD.replace(',', '')}_roc.txt")